<a href="https://colab.research.google.com/github/finiteautomata/ml-examples/blob/master/notebooks/keras/nietzsche_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras LSTM Text generation

We use [this as inspiration](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)

We will generate text trained with Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.


In [1]:

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.


In [2]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
    
print("Length: {}".format(len(text)))

Length: 600893


In [3]:
print(text[:100])

preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all ph


We will be using a char-level LSTM network to generate the text:

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


Our training instances will be pieces of 40 characters, and the target will be the next character. We will select them in a moving-average fashion, with a step of 3 characters.

In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 200285


In [6]:
print("X[0] ---> ", sentences[0])
print("Y[0] ---> ", next_chars[0])

X[0] --->  preface


supposing that truth is a woma
Y[0] --->  n


Now, let's convert this intro a tensor.

We have 200285 sentences of 40 characters each. To convert this into a one-hot representation, we will convert them into a tensor of $200285 \times 40 \times 57$, where 57 is the number of different characters in our text.

Also, $Y$ will be of dimension $200285 \times 57$

In [7]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

   
print("X shape = ", X.shape)
print("Y shape = ", y.shape)

Vectorization...
X shape =  (200285, 40, 57)
Y shape =  (200285, 57)


In [8]:
from keras.layers import CuDNNLSTM
print('Build model...')
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


# Training and Sampling

We will use a couple of strategies to sample random Nietzschean phrases ([see this link for more info](https://medium.com/machine-learning-at-petiteprogrammer/sampling-strategies-for-recurrent-neural-networks-9aea02a6616f))

- Random Sampling: Just sample from the output of the softmax 
- Temperature Sampling: Apply a temperature before sampling

Observation:

Sampling with temperature equal to $1.0$ is the same as sampling from the output of the softmax

In [9]:
  

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    
    for i in range(400):
        # Generate the sentence vector
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
    
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.4)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
200285/200285 [==============================] - 31s 153us/step - loss: 1.9860

----- Generating text after Epoch: 0
----- Generating with seed: "aggot and stake, or of the bull-fight,
t"
aggot and stake, or of the bull-fight,
the form of the somethered more and the
most be one simple consequences as there it delicated and interperting the seesed as the more more one unidigion of
the consequently in the former to the foren the religions of the most as a solitical and for his at the deciesting a the suspections of the fore be made of the self-dount and the seess to the present, and religions of master the most as a most t
Epoch 2/10
200285/200285 [==============================] - 29s 145us/step - loss: 1.6262

----- Generating text after Epoch: 1
----- Generating with seed: "
of flight, a means and artifice for wit"

of flight, a means and artifice for with which is the way and persans and the reflection and the strong and the conscience of the strangle of the mans of the co